# Import

In [1]:
import os
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from scipy import stats
from tensorflow.python.ops import math_ops
from tensorflow.python.keras import backend as K

### add random seed
tf.random.set_seed(3)
# tf.random.set_random_seed(3)

2022-05-14 14:22:23.802891: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/lib:/usr/lib:
2022-05-14 14:22:23.802926: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# 読み込み

In [2]:
%%time
n_features = 300
features = [f'f_{i}' for i in range(n_features)]
feature_columns = ['investment_id', 'time_id'] + features
train = pd.read_parquet('../data/input/ubiquant-parquet/train_low_mem.parquet', columns=feature_columns + ["target"])
train.head()

CPU times: user 5.43 s, sys: 7.05 s, total: 12.5 s
Wall time: 8.44 s


,investment_id,time_id,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,...,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299,target
0,1,0,0.932573,0.113691,-0.402206,0.378386,-0.203938,-0.413469,0.965623,1.230508,...,-1.095620,0.200075,0.819155,0.941183,-0.086764,-1.087009,-1.044826,-0.287605,0.321566,-0.300875
1,2,0,0.810802,-0.514115,0.742368,-0.616673,-0.194255,1.771210,1.428127,1.134144,...,0.912726,-0.734579,0.819155,0.941183,-0.387617,-1.087009,-0.929529,-0.974060,-0.343624,-0.231040
2,6,0,0.393974,0.615937,0.567806,-0.607963,0.068883,-1.083155,0.979656,-1.125681,...,0.912726,-0.551904,-1.220772,-1.060166,-0.219097,-1.087009,-0.612428,-0.113944,0.243608,0.568807
3,7,0,-2.343535,-0.011870,1.874606,-0.606346,-0.586827,-0.815737,0.778096,0.298990,...,0.912726,-0.266359,-1.220772,0.941183,-0.609113,0.104928,-0.783423,1.151730,-0.773309,-1.064780
4,8,0,0.842057,-0.262993,2.330030,-0.583422,-0.618392,-0.742814,-0.946789,1.230508,...,0.912726,-0.741355,-1.220772,0.941183,-0.588445,0.104928,0.753279,1.345611,-0.737624,-0.531940


In [3]:
investment_id = train.pop("investment_id")
investment_id.head()

0    1
1    2
2    6
3    7
4    8
Name: investment_id, dtype: uint16

# 学習済みモデル

In [4]:
# investment_id入力層を定義

# %%time
investment_ids = list(investment_id.unique())
investment_id_size = len(investment_ids) + 1
investment_id_lookup_layer = layers.IntegerLookup(max_tokens=investment_id_size)
# with tf.device("cpu"):
#     investment_id_lookup_layer.adapt(investment_id)
investment_id_lookup_layer.adapt(investment_id)

2022-05-14 14:22:34.062324: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/lib:/usr/lib:
2022-05-14 14:22:34.062359: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-14 14:22:34.062378: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-31-17-131): /proc/driver/nvidia/version does not exist
2022-05-14 14:22:34.062609: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# baseモデル

def get_model():
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
    x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    output = layers.Dense(1)(x)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape"])
    return model

# CNN2Dモデル
def get_model_cnn2d():
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
    ## Dense 1 ##
    feature_x = layers.Dense(256, activation='swish', kernel_initializer = 'he_normal')(features_inputs)
    feature_x = layers.Dropout(0.1)(feature_x)
#     ## Dense 2 ##
#     feature_x = layers.Dense(256, activation='swish')(feature_x)
#     feature_x = layers.Dropout(0.1)(feature_x)
    ## convolution 1 ##
    feature_x = layers.Reshape((-1,1))(feature_x)
    feature_x = layers.Conv1D(filters=16, kernel_size=4, strides=1, padding='same', kernel_initializer = 'he_normal')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU(0.5)(feature_x)
    ## convolution 2 ##
    feature_x = layers.Conv1D(filters=16, kernel_size=4, strides=4, padding='same', kernel_initializer = 'he_normal')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU(0.5)(feature_x)
    ## convolution 3 ##
    feature_x = layers.Conv1D(filters=64, kernel_size=4, strides=1, padding='same', kernel_initializer = 'he_normal')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU(0.5)(feature_x)

    ## convolution2D 1 ##
    feature_x = layers.Reshape((64,64,1))(feature_x)
    feature_x = layers.Conv2D(filters=32, kernel_size=4, strides=1, padding='same', kernel_initializer = 'he_normal')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU(0.5)(feature_x)
    ## convolution2D 2 ##
    feature_x = layers.Conv2D(filters=32, kernel_size=4, strides=4, padding='same', kernel_initializer = 'he_normal')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU(0.5)(feature_x)
    ## convolution2D 3 ##
    feature_x = layers.Conv2D(filters=32, kernel_size=4, strides=4, padding='same', kernel_initializer = 'he_normal')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU(0.5)(feature_x)

    ## flatten ##
    feature_x = layers.Flatten()(feature_x)
    ## Dense 3 ##
    x = layers.Dense(512, activation='swish', kernel_regularizer="l2", kernel_initializer = 'he_normal')(feature_x)
    ## Dense 4 ##
    x = layers.Dropout(0.1)(x)
    ## Dense 5 ##    
    x = layers.Dense(128, activation='swish', kernel_regularizer="l2", kernel_initializer = 'he_normal')(x)
    x = layers.Dropout(0.1)(x)
    ## Dense 6 ##
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2", kernel_initializer = 'he_normal')(x)
    x = layers.Dropout(0.1)(x)
    ## Dense 7 ##
    output = layers.Dense(1)(x)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[features_inputs], outputs=[output])
    
    
    learning_sch = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate = 0.001,
        decay_steps = 9700,
        decay_rate = 0.98
    )
    adam = tf.keras.optimizers.Adam(learning_rate = learning_sch)
    
    
    model.compile(optimizer = adam, loss='mse', metrics=['mse', "mae", "mape", 
#                                                          rmse, correlation
                                                        ])
    return model

In [14]:
!ls ../data/model_wight/ump-conv2d-fold5-outputs/

checkpoint			model_2.tf.index
model_0.tf.data-00000-of-00001	model_3.tf.data-00000-of-00001
model_0.tf.index		model_3.tf.index
model_1.tf.data-00000-of-00001	model_4.tf.data-00000-of-00001
model_1.tf.index		model_4.tf.index
model_2.tf.data-00000-of-00001


In [15]:
model_base = []

for i in range(5):
    model = get_model()
    model.load_weights(f'../data/model_wight/dnn-base-outputs/model_{i}.tf')
    model_base.append(model)

models_cnn2d = []
for i in range(5):
    model = get_model_cnn2d()
    model.load_weights(f'../data/model_wight/ump-conv2d-fold5-outputs/model_{i}.tf')
    models_cnn2d.append(model)

# スコアリング用マート

In [20]:
test_df =pd.read_csv('../data/output/example_test.csv')

In [21]:
test_df.head()

,row_id,time_id,investment_id,f_0,f_1,f_2,f_3,f_4,f_5,f_6,...,f_290,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299
0,1220_1,1220,1,0.874944,0.457875,0.962572,0.372061,0.004509,0.971304,0.097477,...,0.514031,0.828523,0.437697,0.810764,0.693963,0.186403,0.569144,0.120386,0.419603,0.936966
1,1220_2,1220,2,0.554862,0.445478,0.714034,0.891988,0.703331,0.264478,0.600636,...,0.620919,0.301540,0.687464,0.400168,0.908198,0.272901,0.222430,0.748324,0.858802,0.362829
2,1221_0,1221,0,0.175070,0.918464,0.044379,0.505868,0.037160,0.169001,0.211365,...,0.397710,0.789382,0.238078,0.254068,0.795084,0.491928,0.376725,0.079489,0.995041,0.482283
3,1221_1,1221,1,0.355239,0.689773,0.452643,0.134863,0.794958,0.837197,0.036773,...,0.893571,0.322879,0.674692,0.114160,0.818483,0.287065,0.710537,0.550434,0.899869,0.928997
4,1221_2,1221,2,0.912778,0.462684,0.454067,0.491009,0.050547,0.250051,0.686295,...,0.031662,0.703427,0.861476,0.897927,0.659051,0.356148,0.375990,0.209798,0.175153,0.727160


# スコアリング